In [ ]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null && echo "All libraries are successfully installed!"


In [43]:
import os

import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import label2rgb

import albumentations as A
import random

import pandas as pd
import os

In [44]:
source_dir = '/Users/Vaibhav_Beohar/Documents/VB_Mck_Docs/MIDS/W251/w251_mids_summer_2020/final_proj/dataset/nabirds/'
source_dir_images = '/Users/Vaibhav_Beohar/Documents/VB_Mck_Docs/MIDS/W251/w251_mids_summer_2020/final_proj/dataset/nabirds/images/'


# images.txt contains the image and path to the image
df = pd.read_csv(source_dir + 'images.txt', sep=" ", header=None, names=['image_id','path'])
df = df.set_index('image_id')

# bounding_boxes.txt contains the quadrants
# 0000139e-21dc-4d0c-bfe1-4cae3c85c829 83 59 128 228
df_bb = pd.read_csv(source_dir + 'bounding_boxes.txt', sep=" ", header=None, names=["image_id", "x1","y1","x2","y2"])
df_bb = df_bb.set_index('image_id')
df = df.join(df_bb)

# sizes.txt contains the file sizes
# 0000139e-21dc-4d0c-bfe1-4cae3c85c829 296 341
df_sz = pd.read_csv(source_dir + 'sizes.txt', sep=" ", header=None, names = ["image_id", "h","w"])
df_sz = df_sz.set_index('image_id')
df = df.join(df_sz)

# image_class_labels.txt contains the label
df_lb = pd.read_csv(source_dir + 'image_class_labels.txt', sep=" ", header=None, names=["image_id", "label"])
df_lb = df_lb.set_index('image_id')
df = df.join(df_lb)

# first, let's rewrite the classes file in a more consumable csv format
fh = open(source_dir + 'classes.txt','r')
fh2 = open(source_dir + 'classes.csv','w')
for line in fh:
     res = line.rstrip().split(' ')
     img_class = res[0]
     descr = " ".join(res[1:])
     fh2.write(img_class + ",\"" + descr + "\"\n")
fh2.close()
fh.close()

# classes.txt contains the english match
df_en = pd.read_csv(source_dir + 'classes.csv', sep=",", header=None)
df_en.columns = ["label", "english"]
df = df.join(df_en, on='label', how='inner', rsuffix='_')
# df.head()

# convert the image label_ column to string and add the padding zero for directory 
df['label_'] = df['label_'].astype(str)
df['label_'] = df['label_'].apply(lambda x: x.zfill(4))
df = df.rename(columns={"label_": "dir"})

In [45]:
# limiting bird types to 16 primary ones for our focus
df = df[df['english'].str.contains('Blackbird|Warbler|Sparrow|Grackle|Dove|Cardinal|Grosbeak|Chickadee|Goldfinch|Blue Jay|Catbird|Oriole|Wren|Robin|Woodpecker|Thrasher')] 

# mapping all birds to their respective categories
bird_map = {   'Blackbird':'Blackbird',
               'Warbler':'Warbler',
               'Sparrow': 'Sparrow',
               'Grackle': 'Grackle',
               'Dove': 'Dove',
               'Cardinal': 'Cardinal',
               'Grosbeak': 'Grosbeak',
               'Chickadee': 'Chickadee',
               'Goldfinch': 'Goldfinch',
               'Blue Jay': 'Blue Jay',
               'Catbird': 'Catbird',
               'Oriole': 'Oriole',
               'Wren': 'Wren',
               'Robin': 'Robin',
               'Woodpecker': 'Woodpecker',
               'Thrasher': 'Thrasher'
           }

def like_function(x):
    group = "unknown"
    for key in bird_map:
        if key in x:
            group = bird_map[key]
            break
    return group

df['bird_group'] = df.english.apply(like_function)

# new column to check if the directory exists 
df['isdir'] = 'true'

In [46]:
df['isdir']=df['dir'].apply(lambda x: 'true' 
                            if os.path.isdir(source_dir + 'images/'+ x) else 'false')

In [47]:
# display(df)
# df.to_csv(source_dir+'birds.csv')

,path,x1,y1,x2,y2,h,w,label,dir,english,bird_group,isdir
image_id,,,,,,,,,,,,
00019306-9d83-4334-b255-a447742edce3,0900/000193069d834334b255a447742edce3.jpg,174,367,219,378,730,1024,900,0900,Savannah Sparrow,Sparrow,true
0007181f-a727-4481-ad89-591200c61b9d,0900/0007181fa7274481ad89591200c61b9d.jpg,47,194,819,573,1024,819,900,0900,Savannah Sparrow,Sparrow,true
0264f078-40d5-4a19-ace8-f5884e5dd65e,0900/0264f07840d54a19ace8f5884e5dd65e.jpg,179,125,370,483,1024,681,900,0900,Savannah Sparrow,Sparrow,true
0272be79-1a20-4c69-a5dc-f432303d1f40,0900/0272be791a204c69a5dcf432303d1f40.jpg,256,93,279,443,803,604,900,0900,Savannah Sparrow,Sparrow,true
0446ea0e-a763-4d09-adf0-ea384770282a,0900/0446ea0ea7634d09adf0ea384770282a.jpg,23,94,755,640,1024,768,900,0900,Savannah Sparrow,Sparrow,true
...,...,...,...,...,...,...,...,...,...,...,...,...
f62bdcd1-a4b7-4d48-83da-d9c9e01c8727,0881/f62bdcd1a4b74d4883dad9c9e01c8727.jpg,299,200,468,380,800,635,881,0881,Townsend's Warbler,Warbler,true
f6bd3b96-dc40-4e99-8681-bc3a35cf896d,0881/f6bd3b96dc404e998681bc3a35cf896d.jpg,94,120,363,354,1024,683,881,0881,Townsend's Warbler,Warbler,true
f6d609a5-a892-4ce5-9749-85a5c817f157,0881/f6d609a5a8924ce5974985a5c817f157.jpg,121,77,159,282,640,432,881,0881,Townsend's Warbler,Warbler,true


In [125]:

def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)
    
def transform_horiz_flip(new_img_name, new_img_id, image):
    transform = A.HorizontalFlip(p=0.5)
    random.seed(7)
    augmented_image = transform(image=image)['image']
    return new_img_name+'-1.jpg', new_img_id+'-1', augmented_image

def transform_vertical_flip(new_img_name,  new_img_id, image):
    transform = A.VerticalFlip(p=0.5)
    random.seed(100)
    augmented_image = transform(image=image)['image']
    return new_img_name+'-2.jpg',  new_img_id+'-2', augmented_image


# To do:     
    # transform = A.Compose([
    #     A.CLAHE(),
    #     A.RandomRotate90(),
    #     A.Transpose(),
    #     A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
    #     A.Blur(blur_limit=3),
    #     A.OpticalDistortion(),
    #     A.GridDistortion(),
    #     A.HueSaturationValue(),
    # ])
    # random.seed(42) 
    # augmented_image = transform(image=image)['image']
    # visualize(augmented_image)    

In [126]:
transformed_images_df = pd.DataFrame([], columns=df.columns, index=image_id)

In [127]:
list_of_rows = []
list_of_index = []
image_id = df.index.tolist()

for i in range(0, len(df)):
    image = cv2.imread(source_dir_images + df.iloc[i][0])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #visualize(image)  
    
    img = df.iloc[i][0].split('/')
    img_dir = img[0]
    img_name = img[1].split('.')[0]
    
    new_img_name = img_dir + '/'+ img_name + '-' + str(i)
    new_img_id = image_id[i]+ '-' + str(i)
    
    horiz_flip_imgname, horiz_flip_img_id, transformed_img = transform_horiz_flip(new_img_name, new_img_id, image)
    cv2.imwrite(source_dir_images + horiz_flip_imgname, transformed_img)

    vertical_flip_imgname, vertical_flip_img_id, transformed_img = transform_vertical_flip(new_img_name, new_img_id, image)
    cv2.imwrite(source_dir_images + vertical_flip_imgname, transformed_img)
     
    # creating lists for appending into final dataframe
    list_of_rows.append((horiz_flip_imgname, df.iloc[i][1], df.iloc[i][2], df.iloc[i][3], 
                  df.iloc[i][4], df.iloc[i][5], df.iloc[i][6], df.iloc[i][7], df.iloc[i][8], df.iloc[i][9],
                 df.iloc[i][10], df.iloc[i][11]))
    list_of_index.append(horiz_flip_img_id)
    
    list_of_rows.append((vertical_flip_imgname, df.iloc[i][1], df.iloc[i][2], df.iloc[i][3], 
                  df.iloc[i][4], df.iloc[i][5], df.iloc[i][6], df.iloc[i][7], df.iloc[i][8], df.iloc[i][9],
                 df.iloc[i][10], df.iloc[i][11]))
    list_of_index.append(vertical_flip_img_id)
    


In [128]:
dfObj = pd.DataFrame(list_of_rows, columns = df.columns, index=list_of_index) 
# display(dfObj)

In [131]:
df = df.append(dfObj)
df.to_csv(source_dir+'birds.csv')

display(df)

,path,x1,y1,x2,y2,h,w,label,dir,english,bird_group,isdir
00019306-9d83-4334-b255-a447742edce3,0900/000193069d834334b255a447742edce3.jpg,174,367,219,378,730,1024,900,0900,Savannah Sparrow,Sparrow,true
0007181f-a727-4481-ad89-591200c61b9d,0900/0007181fa7274481ad89591200c61b9d.jpg,47,194,819,573,1024,819,900,0900,Savannah Sparrow,Sparrow,true
0264f078-40d5-4a19-ace8-f5884e5dd65e,0900/0264f07840d54a19ace8f5884e5dd65e.jpg,179,125,370,483,1024,681,900,0900,Savannah Sparrow,Sparrow,true
0272be79-1a20-4c69-a5dc-f432303d1f40,0900/0272be791a204c69a5dcf432303d1f40.jpg,256,93,279,443,803,604,900,0900,Savannah Sparrow,Sparrow,true
0446ea0e-a763-4d09-adf0-ea384770282a,0900/0446ea0ea7634d09adf0ea384770282a.jpg,23,94,755,640,1024,768,900,0900,Savannah Sparrow,Sparrow,true
...,...,...,...,...,...,...,...,...,...,...,...,...
f6d609a5-a892-4ce5-9749-85a5c817f157-13318-2,0881/f6d609a5a8924ce5974985a5c817f157-13318-2.jpg,121,77,159,282,640,432,881,0881,Townsend's Warbler,Warbler,true
f7a8cc63-8842-40a4-9ff8-fe530310d62c-13319-1,0881/f7a8cc63884240a49ff8fe530310d62c-13319-1.jpg,275,119,433,528,1024,683,881,0881,Townsend's Warbler,Warbler,true
f7a8cc63-8842-40a4-9ff8-fe530310d62c-13319-2,0881/f7a8cc63884240a49ff8fe530310d62c-13319-2.jpg,275,119,433,528,1024,683,881,0881,Townsend's Warbler,Warbler,true
f8bd891e-aa09-4782-89db-33e7cb2b0cf1-13320-1,0881/f8bd891eaa09478289db33e7cb2b0cf1-13320-1.jpg,38,190,941,493,1024,769,881,0881,Townsend's Warbler,Warbler,true
